# Configure a digital twin: Cloud harvester
This is a HMI which allows to configure a simple digital twin based on cloud approach using python reference implementation and S³I python library. With the following example in terms of harvester's dt, we will show you how to register the identify of a digital twin and create the respective configuration file with this HMI. All necessary modules are imported into the script.   
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I and modelling language library.

In [ ]:
import s3i
import jwt
import getpass
import json 
import os
from ml.tools import load_config, make_thing_config, make_sub_thing
from tools import print_with_timestamp, yes, no, dt_creation_app_id, dt_creation_app_secret

## Configure the notebook
In order to register a thing identify for the harvester, we need a connection with S³I. This means, this notebook needs a client id and the respective secret to authenticate itself at the S³I. The client credentials are already prepared for you. You only need to enter your username and password in the following input fields.

In [ ]:
print_with_timestamp("KWH HMI to configure a digital twin, please log in!")
username = input('[S3I]: Please enter your username:').strip('," ')
password = getpass.getpass('[S3I]: Please enter the password:')
print_with_timestamp("Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', 
                                             identity_provider_url="https://idp.s3i.vswf.dev/",
                                             realm='KWH',
                                             client_id=dt_creation_app_id,
                                             client_secret=dt_creation_app_secret,
                                             username=username,
                                             password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)

''' decode the access token'''
parsed_username = jwt.decode(access_token, verify=False)["preferred_username"]

print_with_timestamp("Token received, " + parsed_username + " logged in.")

## Create an identity for your digital twin
In the following, the identify is created using the S³I Config API. Meanwhile, we also create a S³I-B endpoint in the S³I Broker and a cloud copy in the S³I Repository. 

In [ ]:
s3i_config = s3i.Config(token=access_token)
resp = s3i_config.create_thing()
dt_id = resp.json().get("identifier", None)
dt_secret = resp.json().get("secret", None)
s3i_config.create_cloud_copy(thing_id=dt_id)
print_with_timestamp("Your digital twin has the identifier: {}".format(dt_id))
print_with_timestamp("Your digital twin has the respective secret: {}".format(dt_secret))

## Make the configuration file 
As next we will introduce how to simply configure your digital twin of harvester using python reference implementation library. All the roles and features used have been defined in fml40, and implemented in the corresponding library. 

In [ ]:
%%html
<img src="harvester_images/model_harvester.png", width=500, height=500>

In [ ]:
root_path = os.path.abspath("")

dt_name = input("[S³I]: Please name your digital twin Harvester(e.g. my_cloud_harvester): ")

config_file_name = make_thing_config(dt_id=dt_id, name=dt_name, roles=[{"class": "fml40::Harvester"}],
                                     features=[{"class": "ml40::RotationalSpeed", "rpm": 2005}], root=root_path)

dt_model = load_config(config_file_name, root=root_path)

print_with_timestamp("The date model of the created harvester: " + json.dumps(dt_model, indent=2))
print_with_timestamp("The created configuration file is stored in the folder 'configs'.")